In [ ]:
try:
    import assist
except ImportError:
    from os.path import abspath
    from sys import path
    assist_path = abspath('..')
    if assist_path not in path:
        path.append(assist_path)
    import assist
    del assist_path, abspath
    
del assist

In [ ]:
%pylab inline
from __future__ import division, print_function

from assist.environment import Atmosphere
from assist.aircraft import Aircraft
from assist.component import Wing, Engine, Payload
from assist.mission import Mission, Segment

#Aircraft Sizing, Synthesis and Integration Support Tool (ASSIST)

##References
* Mattingly, J. D., Heiser, W. H., and Daley, D. H., "Aircraft Engine Design," AIAA Education Series, AIAA, New York, NY, 1987.
* Raymer, D. P., "Aircraft Design: A Conceptual Approach," 3rd ed., AIAA Education Series, AIAA, Reston, VA, 1999.
* Roskam, J., "Airplane Design: Part I-VIII." DARcorporation, Lawrence, KS, 2006.

In [ ]:
wing = Wing(flap_type='single_slot',
            configuration='landing',
            slats=True,
            k_aero=0.75,
            sweep=30,
            flap_span=[0.2, 0.4],
            taper_ratio=0.2)

wing.takeoff
print("TAKE-OFF CL_max:", wing.cl_max)
wing.landing
print("LANDING CL_max:", wing.cl_max)

In [ ]:
from numpy import power, array, divide, float
density = 0.002047
k_to = 1.2
cl_max = 1.45
time_rotation = 3
t_to_w = linspace(0.01, 2.0, 200)
tofl = 1500
beta = 1.0
G_0 = 32.17
alpha = 1.0

a = k_to * k_to * beta * beta / (density * G_0 * cl_max * alpha * t_to_w)
b = time_rotation * k_to * sqrt(2 * beta / (density * cl_max))
c = tofl

w_to_s = power((-b + sqrt(b * b + 4 * a * c)) / (2 * a), 2)

In [ ]:
aircraft = Aircraft(wing=wing,
                    stores=[Payload('Crew', weight=200),
                            Payload('Cannon', weight=270),
                            Payload('Ammunition Feed System', weight=405),
                            Payload('Ammunition', weight=550),
                            Payload('Casings', weight=198),
                            Payload('AMRAAMs', weight=332, quantity=4, cd_r=0.005, expendable=True),
                            Payload('AIM-9Xs', weight=188, quantity=2, cd_r=0.002, expendable=True)],
                    drag_chute=None)#{'diameter': 15.6, 'cd': 1.4})

tofl = 1500
airfield_altitude = 0

cruise_altitude = 30000
cruise_range = 150

dash_altitude = 30000
dash_speed = 1492
dash_range = 100

ldgfl = 1500
landing_speed = 150

mission = Mission(segments=[Segment('warmup',
                                    altitude=airfield_altitude,
                                    speed=0,
                                    time=60),
                            Segment('takeoff',
                                    altitude=airfield_altitude,
                                    speed=150,
                                    field_length=tofl,
                                    temperature=100),
                            Segment('climb',
                                    altitude=airfield_altitude,
                                    speed=500),
                            Segment('cruise',
                                    altitude=cruise_altitude,
                                    speed=1492,
                                    range=cruise_range,
                                    release=[()]),
                            Segment('descend',
                                    altitude=dash_altitude,
                                    speed=1000),
                            Segment('dash',
                                    altitude=dash_altitude,
                                    speed=dash_speed,
                                    range=dash_range),
                            Segment('climb',
                                    altitude=cruise_altitude,
                                    speed=1000),
                            Segment('cruise',
                                    altitude=cruise_altitude,
                                    speed=1050,
                                    range=cruise_range),
                            Segment('descend',
                                    altitude=airfield_altitude,
                                    speed=1000),
                            Segment('land',
                                    altitude=airfield_altitude,
                                    speed=landing_speed,
                                    field_length=ldgfl)])

In [ ]:
%pdb on
aircraft._synthesize(mission)

In [ ]:
aircraft._k_1

In [ ]:
fig = figure(figsize=(16, 8))
w_s = aircraft._synthesis['w_to_s']
t_w = aircraft._synthesis['t_to_w']
for t in t_w:
    plot(w_s, t)
    legend([segment.kind for segment in mission.segments])
    
axis([0, 300, 0, 3.5]);

In [ ]:
fig = figure(figsize=(16, 8))
ax = plot(aircraft.t_to_w_req)
axis([0, 300, 0, 5])

In [ ]:
aircraft.fuel_fraction

In [ ]:
aircraft._size(mission)

# Aircraft Dimensions

In [ ]:
# Takeoff Gross Weight
aircraft.w_to

# Engine Dimensions

In [ ]:
array([[1,2],[2,3]])[0]

In [ ]:
aircraft.engine.size()
aircraft.engine.__dict__

In [ ]:
aircraft._k_1 = None

In [ ]:
aircraft.best_cruise